<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_Single_Trial_Classification_PatientData_CrossSubject_Grid_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/EEG_Deep.git

Cloning into 'EEG_Deep'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 412 (delta 62), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (412/412), 1.69 GiB | 36.62 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Checking out files: 100% (97/97), done.


In [0]:
import scipy.io as spio
import numpy as np
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

from numpy import array
from sklearn.model_selection import KFold

In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [4]:
from numpy import zeros
K.clear_session()

h_cut = [24]
drop_out = [0.5]
k_len = [32, 64, 128]
n_epochs = 500

nsub = 10
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
acc_sub = zeros([nsub, nfilt,ndrop,nkl])

outfname = 'accuray_epochs' + str(n_epochs) + '_filter_' + str(h_cut) + '_drop_' + str(drop_out) + '_Cross_patient_data.npy'

result=[]
count = 0
# data sample
data = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# prepare cross validation
kfold = KFold(10, True, 1)
# enumerate splits
for train, test in kfold.split(data):
  count = count + 1
  # print('train: %s, test: %s' % (data[train], data[test]))
  r_X_tr = np.empty([0, 12, 4096])
  Y_tr = np.empty([0,1])
  r_X_ts = np.empty([0, 12, 4096])
  Y_ts = np.empty([0,1])
  X_tr = np.empty([720, 12, 4096])
  X_ts = np.empty([40, 12, 4096])
  

  for x in data[train]:
    print(x)
    fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'T.mat'  # Load Data
    print(fName)
    mat = spio.loadmat(fName)
    x_tr = mat['RawEEGData']
    y_tr = mat['Labels']
    print(r_X_tr.shape)
    r_X_tr=np.append(r_X_tr, x_tr, axis=0)
    Y_tr=np.append(Y_tr, y_tr, axis=0)
    print(r_X_tr.shape)
    print(Y_tr.shape)


  for x in data[test]:
    print(x)
    subid = x 
    fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'E.mat'  # Load Data
    print(fName)
    mat = spio.loadmat(fName)
    x_ts = mat['RawEEGData']
    y_ts = mat['Labels']
    print(r_X_ts.shape)
    r_X_ts=np.append(r_X_ts, x_ts, axis=0)
    Y_ts=np.append(Y_ts, y_ts, axis=0)
    print(r_X_ts.shape)
    print(Y_ts.shape)

    for h_indx, h in enumerate(h_cut):
      ### Filter Training Data ###
      print("Filtering of training data in progress")
      for t in range(r_X_tr.shape[0]):
        tril = r_X_tr[t,:,:]
        # tril = tril.transpose()
        tril_filtered = butter_bandpass_filter(tril, lowcut=8,
                                               highcut=h, fs=512,
                                               order=4)
        # tril_filtered = tril_filtered.transpose()
        X_tr[t,:,:] = tril_filtered
  
      print(X_tr.shape)

      ### Filter Test Data Data ###
      print("Filtering of test data in progress")
      for t in range(r_X_ts.shape[0]):
        tril = r_X_ts[t,:,:]
        # tril = tril.transpose()
        tril_filtered = butter_bandpass_filter(tril, lowcut=8, 
                                               highcut=h, fs=512,
                                               order=4)
        # tril_filtered = tril_filtered.transpose()
        X_ts[t,:,:] = tril_filtered

    print(X_ts.shape)

    indices = np.arange(X_tr.shape[0])
    np.random.shuffle(indices)

    X_tr = X_tr[indices]
    Y_tr = Y_tr[indices]

    # split data of each subject in training and validation
    X_train = X_tr[0:620,:,2560:4096]
    Y_train = Y_tr[0:620].ravel()
    X_val   = X_tr[620:,:,2560:4096]
    Y_val   = Y_tr[620:].ravel()
    # print(Y_val)
    print(np.shape(X_train))
    print(np.shape(Y_train))
    print(np.shape(X_val))
    print(np.shape(Y_val))

    # convert labels to one-hot encodings.
    Y_train      = np_utils.to_categorical(Y_train-1, num_classes=2)
    Y_val       = np_utils.to_categorical(Y_val-1, num_classes=2)
    # print(Y_val)

    kernels, chans, samples = 1, 12, 1536
    # convert data to NCHW (trials, kernels, channels, samples) format. Data 
    # contains 22 channels and 500 time-points. Set the number of kernels to 1.
    X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
    X_val   = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_val.shape[0], 'val samples')

    X_test      = X_ts[:,:,2560:4096]
    Y_test      = Y_ts[:]
    print(np.shape(X_test))
    print(np.shape(Y_test))

    #convert labels to one-hot encodings.
    Y_test      = np_utils.to_categorical(Y_test-1, num_classes=2)

    # convert data to NCHW (trials, kernels, channels, samples) format. Data 
    # contains 22 channels and 500 time-points. Set the number of kernels to 1.
    X_test = X_test.reshape(X_test.shape[0], kernels, chans, samples)
    print('X_train shape:', X_test.shape)
    print(X_test.shape[0], 'train samples')

    for id_d, d in enumerate(drop_out):
      for id_kl, kl in enumerate(k_len):
        print(id_kl, id_d)
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = EEGNet(nb_classes = 2, Chans = 12, Samples = 1536,
                       dropoutRate = d, kernLength = kl, F1 = 8, 
                       D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
      
        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', metrics = ['accuracy'])
        
        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {0:1, 1:1}

        history = model.fit(X_train, Y_train, batch_size = 16, epochs = n_epochs, 
                            verbose = 2, validation_data=(X_val, Y_val),
                            callbacks=[checkpointer], class_weight = class_weights)
        
        print('\n# Evaluate on test data')
        results = model.evaluate(X_test, Y_test, batch_size=1)
        print('test loss, test acc:', results)

        acc_sub[subid-1,h_indx,id_d, id_kl] = results[1]

        from keras import backend as K 
        # Do some code, e.g. train and save model
        K.clear_session()


      
np.save(outfname, acc_sub)
    

1
EEG_Deep/Data2A/parsed_P01T.mat
(0, 12, 4096)
(80, 12, 4096)
(80, 1)
2
EEG_Deep/Data2A/parsed_P02T.mat
(80, 12, 4096)
(160, 12, 4096)
(160, 1)
4
EEG_Deep/Data2A/parsed_P04T.mat
(160, 12, 4096)
(240, 12, 4096)
(240, 1)
5
EEG_Deep/Data2A/parsed_P05T.mat
(240, 12, 4096)
(320, 12, 4096)
(320, 1)
6
EEG_Deep/Data2A/parsed_P06T.mat
(320, 12, 4096)
(400, 12, 4096)
(400, 1)
7
EEG_Deep/Data2A/parsed_P07T.mat
(400, 12, 4096)
(480, 12, 4096)
(480, 1)
8
EEG_Deep/Data2A/parsed_P08T.mat
(480, 12, 4096)
(560, 12, 4096)
(560, 1)
9
EEG_Deep/Data2A/parsed_P09T.mat
(560, 12, 4096)
(640, 12, 4096)
(640, 1)
10
EEG_Deep/Data2A/parsed_P010T.mat
(640, 12, 4096)
(720, 12, 4096)
(720, 1)
3
EEG_Deep/Data2A/parsed_P03E.mat
(0, 12, 4096)
(40, 12, 4096)
(40, 1)
Filtering of training data in progress
(720, 12, 4096)
Filtering of test data in progress
(40, 12, 4096)
(620, 12, 1536)
(620,)
(100, 12, 1536)
(100,)
X_train shape: (620, 1, 12, 1536)
620 train samples
100 val samples
(40, 12, 1536)
(40, 1)
X_train shape: 

Using TensorFlow backend.


Train on 620 samples, validate on 100 samples
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.68160, saving model to /tmp/checkpoint.h5
620/620 - 1s - loss: 0.6937 - acc: 0.5194 - val_loss: 0.6816 - val_acc: 0.5500
Epoch 2/500

Epoch 00002: val_loss improved from 0.68160 to 0.66737, saving model to /tmp/checkpoint.h5
620/620 - 1s - loss: 0.6671 - acc: 0.6113 - val_loss: 0.6674 - val_acc: 0.6200
Epoch 3/500

Epoch 00003: val_loss improved from 0.66737 to 0.63573, saving model to /tmp/checkpoint.h5
620/620 - 1s - loss: 0.6351 - acc: 0.6645 - val_loss: 0.6357 - val_acc: 0.6900
Epoch 4/500

Epoch 00004: val_loss improved from 0.63573 to 0.60910, saving model to /tmp/checkpoint.h5
620/620 - 1s - loss: 0.6081 - acc: 0.6823 - val_loss: 0.6091 - val_acc: 0.6900
Epoch 5/500

Epoch 00005: val_loss improved from 0.60910 to 0.59379, saving model to /tmp/checkpoint.h5
620/620 - 1s - loss: 0.5742 - acc: 0.7161 - val_loss: 0.5938 - val_acc: 0.7300
Epoch 6/500

Epoch 00006: val_loss improved